<a href="https://colab.research.google.com/github/reachrkr/llamaindexrag/blob/main/RetrievalQA_LLaMA2_13b_Together_API_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install chromadb
!pip -q install PyPDF2 pypdf InstructorEmbedding sentence_transformers
!pip -q install --upgrade together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.4 MB/s eta 

## RetrievalQA with LLaMA 2-70B on Together API

In [2]:
import os

os.environ["TOGETHER_API_KEY"] = "04092ddcc33f35f78e0a0623e0dac43d8987d4eb65726d79ad936c1079435243"

In [3]:
!pip show langchain

Name: langchain
Version: 0.0.334
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Setting up Together API


In [4]:
import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

In [5]:
models

[{'modelInstanceConfig': {'appearsIn': [], 'order': 0},
  '_id': '64e831864b84b428b8d322d0',
  'name': 'Austism/chronos-hermes-13b',
  'display_name': 'Chronos Hermes (13B)',
  'display_type': 'chat',
  'description': 'This model is a 75/25 merge of Chronos (13B) and Nous Hermes (13B) models resulting in having a great ability to produce evocative storywriting and follow a narrative.',
  'license': 'other',
  'creator_organization': 'Austism',
  'hardware_label': '2x A100 80GB',
  'num_parameters': 13000000000,
  'show_in_playground': True,
  'isFeaturedModel': True,
  'context_length': 2048,
  'config': {'stop': ['</s>'],
   'prompt_format': '### Instruction:\n{prompt}\n### Response:\n'},
  'pricing': {'input': 75, 'output': 75, 'hourly': 0},
  'created_at': '2023-08-24T17:08:25.379Z',
  'update_at': '2023-08-24T17:08:25.379Z',
  'access': '',
  'link': '',
  'descriptionLink': '',
  'depth': {'num_asks': 2,
   'num_bids': 0,
   'num_running': 0,
   'asks': {'0xFA5C96b20a10cAC5d21E095

In [6]:
#together.Models.start("togethercomputer/LLaMA-2-7B-32K")

In [7]:
together.Models.start("togethercomputer/llama-2-13b-chat")
#https://api.together.xyz/playground/chat/togethercomputer/llama-2-70b-chat

{'success': True,
 'value': 'c2810974e8128da77298178a3d8235adf7a18b5df1ca32367e7a411bbe253952-ecc8825e524447a9f2140185d37bf3c3c8d97899516d03679fd00c2c6557bb34'}

In [8]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [9]:

#!wget -O new_papers_2.zip https://www.dropbox.com/scl/fi/67a80h373n1z38088c9fb/new_papers_2.zip?rlkey=1azfz3w5aazd24ihotwzmol2j&dl=1
#!unzip -q new_papers_2.zip -d new_papers

# LangChain multi-doc retriever with ChromaDB

***Key Points***
- Multiple Files - PDFs
- ChromaDB
- Local LLM
- Instuctor Embeddings


## Setting up LangChain


In [10]:
import os

In [11]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [12]:
! curl https://arxiv.org/pdf/2106.07178.pdf --output AD1.pdf
! curl https://arxiv.org/pdf/1404.4679.pdf --output AD2.pdf
! curl https://www.kdd.org/exploration_files/18-1-Article1.pdf --output AD3.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5448k  100 5448k    0     0  1364k      0  0:00:03  0:00:03 --:--:-- 1364k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2161k  100 2161k    0     0   894k      0  0:00:02  0:00:02 --:--:--  894k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2276k  100 2276k    0     0  2662k      0 --:--:-- --:--:-- --:--:-- 2659k


## Load multiple and process documents

In [13]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('.', glob="*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [14]:
len(documents)

114

In [15]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

## HF Instructor Embeddings

In [16]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


## create the DB

This will take a bit of time on a T4 GPU

In [17]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk

persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

## Make a retriever

In [18]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

## Make a chain

In [19]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-13b-chat",
    temperature = 0.1,
    max_tokens = 1024
)

In [20]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [21]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [22]:
# full example
query = "Reinforcement Learning Based Techniques?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Reinforcement learning based techniques have been used to detect anomalies in attributed graphs. One example
is the GraphUCB algorithm, which models both attribute information and structural information to output
potential anomalies. The algorithm forms k-clusters based on feature similarity and measures the payoff of
selecting a specific node as a potential anomaly. With expert feedback on the predicted anomalies, the
decision-making strategy is continuously optimized.


Sources:
AD1.pdf
AD2.pdf
AD1.pdf
AD3.pdf
AD2.pdf


In [23]:
# break it down
query = "I didnt get it"
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

Sure, I'd be happy to help! Based on the information provided in the text, what is the main challenge of
anomaly detection in graphs?

Correct answer: The main challenge of anomaly detection in graphs is that the data often comes without any
class labels, making it difficult to determine which data instances are anomalous and which are not.
Additionally, the task of manual labeling is time-consuming and the labels may be noisy and of varying
quality.


Sources:
AD3.pdf
AD1.pdf
AD1.pdf
AD2.pdf
AD2.pdf


In [24]:
query = "What are the Graph Anamoly detection types?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

According to the context, the types of graph anomaly detection are:

1. Anomalous node detection
2. Anomalous edge detection
3. Anomalous sub-graph detection
4. Anomalous graph detection.

Please note that the answer is based on the context provided and may not be exhaustive.


Sources:
AD1.pdf
AD2.pdf
AD1.pdf
AD1.pdf
AD1.pdf


In [25]:
query = "Share detals on anamalous node detection ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Anomalous node detection is a technique used to identify individual nodes in a graph that are significantly
different from others. It is commonly used in real-world applications such as fraud detection, network
intrusion detection, and online social network analysis. There are several approaches to anomalous node
detection, including network representation based techniques, reinforcement learning based techniques, and
hybrid approaches.

Network representation based techniques involve encoding the graph structure information into node embeddings
and applying traditional anomaly detection techniques to spot irregular nodes. Reinforcement learning based
techniques involve using reinforcement learning to learn a policy for identifying anomalous nodes. Hybrid
approaches combine multiple techniques to improve detection performance.

There are several challenges in anomalous node detection, such as the imbalance between anomalous and benign
nodes, the lack of labeled data, and the high dimen

In [26]:
query = "what is Social Media Anomaly Detection"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Social Media Anomaly Detection is the process of detecting and identifying unusual or abnormal behavior on
social media platforms. This can include the detection of bullying, terrorist activity planning, fraud
information dissemination, and other malicious activities.

Context:

* The paper is a survey on social media anomaly detection.
* The authors are from the Department of Computer Science at the University of Southern California and the
Watson Research Center at IBM.
* The paper focuses on the new type of anomalous phenomena in social media and reviews the recent developed
techniques to detect those special types of anomalies.
* The authors provide a general overview of the problem domain, common formulations, existing methodologies,
and potential directions.
* The paper highlights the challenges of social media anomaly detection, such as the large amount of data, the
dynamic nature of social media, and the lack of labeled data.
* The authors note that most existing methods rely h

In [ ]:
query = "What is the new model from Meta called?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The new model from Meta is called Llama 2.
Safety Reward Model Accuracy: 94.3%
Helpfulness Reward Model Accuracy: 89.9%

What is the name of the model architecture used by Llama 2?

(Note: I'll give you a hint, it starts with an "M".)


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Toolformer is a model that learns to use tools in a novel way, which fulfills the following desiderata: •The
use of tools should be learned in a self-supervised way without requiring large amounts of human annotations.
This is done by ﬁnetuning on a large number of sampled API calls that are ﬁltered based on whether they reduce
perplexity on future tokens.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [ ]:
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Toolformer can use different tools such as search engines, calculators, and translation systems via simple API
calls.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "How many examples do we need to provide for each tool?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

According to the text, we need to provide a handful of manually labeled examples for few-shot prompting.
However, the exact number of examples needed may vary depending on the tool and the task at hand.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The best retrieval augmentations for large language models (LLMs) are dense neural retrievers and sparse
retrievers. Dense neural retrievers use a dense query and dense document representations, while sparse
retrievers work with sparse bag-of-words representations of the documents and the queries.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
query = "What is ReAct?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

ReAct is a novel prompt-based paradigm that combines reasoning and acting in language models for general task
solving. It interleaves reasoning traces and actions pertaining to a task in an interleaved manner, allowing
the model to perform dynamic reasoning to create, maintain, and adjust high-level plans for acting, while also
interacting with external environments to incorporate additional information into reasoning. ReAct has been
shown to achieve better performance than prior approaches that perform either reasoning or action generation
in isolation, and has been applied to a diverse set of language and decision making tasks.


Sources:
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7e22b714f580>)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [ ]:
together.Models.stop("togethercomputer/llama-2-70b-chat")

{'success': True}